<a href="https://colab.research.google.com/github/BrownVinci/Video-Compressor/blob/main/Video_Compressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **<- Click Play Icon to Start** { vertical-output: true, display-mode: "form" }
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3><b>Mount GDrive to /content/drive</b><br><i>Connect Drive to Colab</i></h3></center><br>

MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

In [ ]:
# @title **Install All Dependants**
# @markdown <br><center><img src='https://raw.githubusercontent.com/sudo-ken/FFmpeg-for-GDrive/master/res/ffmpeg.png' height="50" alt="Gdrive-logo"/></center>
# @markdown <center><h3><b>Install FFmpeg</b><br><i>It may take 150s or more</i></h3></center><br>

from IPython.display import clear_output
import os
import urllib.request
import time

start_time = time.time()

HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/sudo-ken/FFmpeg-for-GDrive/master/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import loadingAn, textAn

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')

os.system('apt-get install ffmpeg')
os.system('apt-get install -y libass-dev')
os.system('apt-get update')
os.system('apt-get install -y autoconf automake build-essential cmake git libass-dev libfreetype6-dev '
          'libsdl2-dev libtool libva-dev libvdpau-dev libvorbis-dev libxcb1-dev libxcb-shm0-dev '
          'libxcb-xfixes0-dev pkg-config texinfo wget zlib1g-dev')

os.system('git clone https://github.com/FFmpeg/FFmpeg.git')
os.chdir('FFmpeg')
os.system('./configure --enable-libass')
os.system('make -j8')
os.system('make install')

os.system('pip install ffmpeg-python')

end_time = time.time()

clear_output()
print('Installation finished.')
elapsed_time = end_time - start_time
print(f'Total time elapsed: {elapsed_time:.2f} seconds')

In [ ]:
# @title **Get File Media Info**

use_dir = "MyDrive" # @param ["MyDrive", "Shareddrives"]
file_path = "Movies/Jawan (2023) EXTENDED CUT 1080p HDRip x264 AAC 5.1 ESubs [3.2GB].mkv" # @param {type:"string"}

path = f"/content/drive/{use_dir}/{file_path}"
save_txt = False #@param {type:"boolean"}
import os, uuid, re, IPython
import ipywidgets as widgets
import time

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output, drive

def mediainfo():
  display(HTML("<br>"))
#   print(path.split("/")[::-1][0])
  display(HTML("<br>"))
#   media = !mediainfo "$path"
#   media = "\n".join(media).replace(os.path.dirname(path)+"/", "")
  get_ipython().system_raw("""mediainfo --LogFile="/root/.nfo" "$path" """)
  with open('/root/.nfo', 'r') as file:
    media = file.read()
    media = media.replace(os.path.dirname(path)+"/", "")
  print(media)
  get_ipython().system_raw("rm -f '/root/.nfo'")

  if save_txt:
    txt = path.rpartition('.')[0] + ".txt"
    if os.path.exists(txt):
      get_ipython().system_raw("rm -f '$txt'")
    with open(txt, 'a+') as file:
      file.write(media)

while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output()

if not os.path.exists("/usr/bin/mediainfo"):
  get_ipython().system_raw("apt-get install mediainfo")

mediainfo()

In [ ]:
# @title **Video Compressor**

use_dir = "MyDrive" # @param ["MyDrive", "Shareddrives"]
file_path = "Movies/Jawan (2023) EXTENDED CUT 1080p HDRip x264 AAC 5.1 ESubs [3.2GB].mkv" # @param {type:"string"}
resolution = "720p" # @param ["240p", "360p", "480p", "720p", "1080p"]
audio_codec = "aac" # @param ["aac","ace","adts","aiff","ape","aptx","aptx_hd","ast","avi","avr","binka","boa","caf","cavsvideo","epaf","jack","loas","mca","oga","tta","wav","aea","alsa","daud","mp2","mp3","oma","openal","pulse","sndio","wsaud","wve"]
audio_bitrate = "192" # @param ["64", "96", "128", "160", "192", "224", "256", "320", "384", "448", "512", "576", "640", "768", "800"]
audio_channel = "2" # @param ["1","2","3","4","5","6"]
audio_sample_rate = "48000" # @param ["8000", "16000", "22050", "32000", "44100", "48000", "88200", "96000", "176400", "192000", "384000"]
video_codec = "libx264" # @param ["h264","hevc","libx264","libx265","mpeg4","avi","cavsvideo","dhav","dv","f4v","flv","gdv","ipu","ivr","live_flv","luodat","mlv","nsv","nuv","ogv","video4linux2","v4l2","xv","a64","avs2","cavsvideo","cdxl","iv8","m4v","mjpeg","mjpeg_2000","mpeg1video","mpeg2video","mpegvideo","rawvideo","ser","vc1","video4linux2","v4l2"]
video_crf = 22 # @param {type:"integer"}
video_profile = "main" # @param ["baseline", "main", "high", "high10", "high422", "high444", "constrained_baseline", "scalable_baseline", "extended", "scalable_high", "stereo_high", "multiview_high"]
video_preset = "medium" # @param ["ultrafast", "superfast", "veryfast", "faster", "fast", "medium", "slow", "slower", "veryslow", "placebo"]
output_extension = "mp4" # @param ["mkv", "mp4", "avi"]
print_result_every = 10 # @param {type:"integer"}

# @markdown ---
# @markdown * <b>use_dir:</b> The directory to use, either "MyDrive" or "Shareddrives".
# @markdown * <b>file_path:</b> The path to the video file you want to compress.
# @markdown * <b>resolution:</b> The desired resolution for the compressed video, options include <code>"240p," "360p," "480p," "720p," or "1080p"</code>.
# @markdown * <b>audio_codec:</b> The audio codec to use.
# @markdown * <b>audio_bitrate:</b> The bitrate for the audio, specified in kilobits per second.
# @markdown * <b>audio_channel:</b> The number of audio channels.
# @markdown * <b>audio_sample_rate:</b> The audio sample rate, specified in hertz.
# @markdown * <b>video_codec:</b> The video codec to use.
# @markdown * <b>video_crf:</b> Lower CRF values result in higher quality and larger file sizes, while higher CRF values yield lower quality and smaller file sizes. <code>{01~51} int</code>
# @markdown * <b>video_profile:</b> Specifies a predefined set of encoding parameters, such as <code>"main"</code>, affecting compression and compatibility.
# @markdown * <b>video_preset:</b> Sets the encoding preset, influencing the balance between speed and compression efficiency.
# @markdown * <b>output_extension:</b> The desired file extension for the compressed video.
# @markdown * <b>Output or Your File:</b> The compressed video will be automatically saved in the <code>"Compressed"</code> folder within the same directory as the original video.
# @markdown * <b>Est. Remaining:</b> It is not accurate, sorry 😅
# @markdown * <b>Made by:</b> <code>@BrownVinci</code>
# @markdown ---

#Main Code Start form Here

import os
import subprocess
import re
import time

input_file = f"/content/drive/{use_dir}/{file_path}"

# Output folder handling
output_folder = os.path.join(os.path.dirname(input_file), "Compressed")
os.makedirs(output_folder, exist_ok=True)

# Output file name
output_file = f"{output_folder}/{os.path.basename(file_path).split('.')[0]} [{resolution}-{video_codec}-{audio_codec}].{output_extension}"

# Resolution handling
if resolution == "240p":
    resolution_str = "426:240"
elif resolution == "360p":
    resolution_str = "640:360"
elif resolution == "480p":
    resolution_str = "854:480"
elif resolution == "720p":
    resolution_str = "1280:720"
elif resolution == "1080p":
    resolution_str = "1920:1080"
else:
    print("Invalid resolution option.")
    exit(1)
def file_exists(output_file):
    return os.path.exists(output_file)

def confirm_overwrite(output_file):
    while True:
        response = input(f"File '{output_file}' already exists. \nDo you want to overwrite it? (Y/N): ").strip().lower()
        if response in ['y', 'yes']:
            return True
        elif response in ['n', 'no']:
            return False
        else:
            print("Invalid response. Please enter 'Y' or 'N'.")
def run_ffmpeg_with_progress(command):
    last_printed_progress = -1
    last_update_time = time.time()
    process_start_time = last_update_time

    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        shell=True
    )

    total_seconds = None

    for line in process.stdout:
        line = line.strip()

        if line.startswith('Duration'):
            duration_str = line.split(',')[0].split(':')[1:]
            if len(duration_str) >= 3:
                total_seconds = int(duration_str[0]) * 3600 + int(duration_str[1]) * 60 + float(duration_str[2])

        if line.startswith('frame='):
            if total_seconds:
                current_time = line.split('time=')[1].split()[0]
                current_time = current_time.split(':')
                current_seconds = int(current_time[0]) * 3600 + int(current_time[1]) * 60 + float(current_time[2])
                elapsed_time = time.time() - process_start_time
                progress = (current_seconds / total_seconds) * 100

                if time.time() - last_update_time >= print_result_every and abs(progress - last_printed_progress) > 0.01:
                    bar_length = 50
                    num_bars = int(bar_length * progress / 100)
                    progress_bar = '[' + '\033[94m#\033[0m' * num_bars + '-' * (bar_length - num_bars) + ']'

                    remaining_seconds = total_seconds - current_seconds
                    est_remaining_time = time.strftime('%H:%M:%S', time.gmtime(remaining_seconds))
                    total_time = time.strftime('%H:%M:%S', time.gmtime(elapsed_time))

                    frame_info = "|".join(line.split('|')[:7]).strip()

                    # Print progress and other information on separate lines
                    frame_info = frame_info.replace(" fps=", " | fps= ").replace(" q=", " | q= ").replace(" size=", " | size= ").replace(" time=", " | time= ").replace(" bitrate=", " | bitrate= ").replace(" speed=", " | speed= ")
                    print(f"\033[94mProgress:\033[0m {progress:05.2f}% {progress_bar} | Elapsed= {total_time} | Est. Remaining= {est_remaining_time} | {frame_info}")

                    last_update_time = time.time()
                    last_printed_progress = progress

    process.wait()

command = (
    f"ffmpeg -i \"{input_file}\" -c:v {video_codec} -preset {video_preset} -crf {video_crf} -profile:v {profile} "
    f"-vf \"scale={resolution_str}\" -c:a {audio_codec} -b:a {audio_bitrate}k -ac {audio_channel} "
    f"-ar {audio_sample_rate} -strict experimental \"{output_file}\""
    )

print(
      f"\033[94mInput Video:\033[0m {input_file} \n" \
      f"\033[94mOutput Video:\033[0m {output_file} \n" \
      f"\033[94mEnd-Code:\033[0m {command}\n"
      f"\033[94mProgress:\033[0m 00.00% [ The result will show every {print_result_every}sec ]"
      )

# Check if the output file already exists
if file_exists(output_file):
    if not confirm_overwrite(output_file):
        print("Operation canceled. Please choose a different output file name.")
    else:
        print("Overwriting the existing file...")
        run_ffmpeg_with_progress(command)
else:
    run_ffmpeg_with_progress(command)
    print(f"\033[94mProgress:\033[0m 100.0% [ Progress Successfully Completed ]")

In [ ]:
# @title **Command Check**
# @markdown <br><center><img src='https://raw.githubusercontent.com/sudo-ken/FFmpeg-for-GDrive/master/res/ffmpeg.png' height="50" alt="Gdrive-logo"/></center>
# @markdown <center><h3>FFmpeg Commands</h3></center><br>

import ffmpeg

command = "echo \"[\"$(ffmpeg -formats | grep video | awk '{print $2}' | tr ',' '\\n' | sed 's/ //g' | sed 's/^/\"/' | sed 's/$/\"/' | paste -sd, -)\"]\"" #@param {type:"string"}
!{command}

# **Find me on**<br>
- Telegram : [@BrownVinci](https://t.me/BrownVinci)
- Github : [@BrownVinci](https://github.com/BrownVinci)
- Telegram Support Group : [@EncoderSupport](https://t.me/EncoderSupport)
### <br>**Before you leave please make sure to Disconnect and delete the runtime**